# Trabalhando com documentos PDF

* PDF = *Portable Document Format*

---

**Desafios**:

* Trabalhar com arquivos binários mais complexos que os arquivos em formato texto simples;


* Além do texto, esses arquivos armazenam diversas informações sobre fonte, cor e *layout*.

---

**Objetivos**:

* Ler conteúdo textual dos PDFs;


* Compor novos PDFs a partir de documentos existentes.

### 1. Importando as bibliotecas necessárias

In [1]:
import PyPDF2

### 2. Extraindo texto de PDFs

#### 2.1. Lendo o arquivo para a extração de informações

In [2]:
pdfFileObj = open('data/meetingminutes.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

#### 2.2 Verificando o número de páginas

In [3]:
pdfReader.numPages

19

#### 2.3. Extraindo o texto da primeira página

* `getPage(0)` cria um objeto `Page` referente a primeira página do arquivo PDF.


* Obs.: o **PyPDF2** utiliza um índice baseado em zero para obter as páginas.

In [4]:
pageObj = pdfReader.getPage(0)
pageObj.extractText()

'OOFFFFIICCIIAALL  BBOOAARRDD  MMIINNUUTTEESS   Meeting of \nMarch 7\n, 2014\n        \n     The Board of Elementary and Secondary Education shall provide leadership and \ncreate policies for education that expand opportunities for children, empower \nfamilies and communities, and advance Louisiana in an increasingly \ncompetitive glob\nal market.\n BOARD \n of ELEMENTARY\n and \n SECONDARY\n EDUCATION\n  '

### 3. Descriptografando PDFs

* Obs.: a senha do PDF utilizado neste exemplo é **rosebud**.

#### 3.1. Lendo o arquivo para a extração de informações

In [5]:
pdfFileObj = open('data/encrypted.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

#### 3.2. Verificando se o arquivo está criptografado

In [6]:
pdfReader.isEncrypted

True

#### 3.3. Tentando extrair informações do arquivo criptografado

* Qualquer tentativa de chamar um método que leia o arquivo antes que ele seja descriptografado com a senha correta resultará em **erro**.

In [7]:
pdfReader.getPage(0)

PdfReadError: file has not been decrypted

#### 3.4. Repetindo a operação após descriptografar

* Se a senha incorreta for especificada, o método `decrypt()` retornará `0`.

In [8]:
pdfReader.decrypt('rosebud')

1

In [9]:
pageObj = pdfReader.getPage(0)

IndexError: list index out of range

* Obs.: utilizar `getPage()` antes de `decrypt()` ocasiona erro na utilização de `getPage()` após o `decrypt()`.

#### 3.5. Forma correta de realizar a operação

In [10]:
pdfFileObj = open('data/encrypted.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

if pdfReader.isEncrypted:
    pdfReader.decrypt('rosebud')
    pageObj = pdfReader.getPage(0)

* Obs: o método `decrypt()` descriptografa somente o objeto `PdfFileReader`, e não o arquivo PDF propriamente dito.

### 4. Criando PDFs

* Agora vamos utilizar o `PdfFileWriter`.


* O `PyPDF2` não pode escrever qualquer texto em um PDF. As capacidades de escrita de PDF do `PyPDF2` estão limitadas a:
    * Copiar páginas de outros PDFs;
    * Fazer rotação de páginas;
    * Sobrepor páginas;
    * Criptografar arquivos.
    
    
* O `PyPDF2` não permite editar diretamente um PDF. Em vez disso, você deverá criar um novo PDF e então copiar o conteúdo de um documento existente.


* Para criar o arquivo PDF chame o método `write()` de `PdfFileWriter`.

#### 4.1. Copiando páginas

In [11]:
pdf1File = open('data/meetingminutes.pdf', 'rb')
pdf2File = open('data/meetingminutes.pdf', 'rb')

pdf1Reader = PyPDF2.PdfFileReader(pdf1File)
pdf2Reader = PyPDF2.PdfFileReader(pdf2File)

pdfWriter = PyPDF2.PdfFileWriter()

for pageNum in range(pdf1Reader.numPages):
    pageObj = pdf1Reader.getPage(pageNum)
    pdfWriter.addPage(pageObj)
    
    
for pageNum in range(pdf2Reader.numPages):
    pageObj = pdf2Reader.getPage(pageNum)
    pdfWriter.addPage(pageObj)
    
pdfOutputFile = open('data/combinedminutes.pdf', 'wb')
pdfWriter.write(pdfOutputFile)

pdfOutputFile.close()
pdf1File.close()
pdf2File.close()

* Obs.: o método `addPage()` adicionará páginas somente no final do PDF.

#### 4.2. Rotação de páginas

* O PDF resultante terá uma página girada em 90 graus no sentido horário.

In [12]:
minutesFile = open('data/meetingminutes.pdf', 'rb')
pdf1Reader = PyPDF2.PdfFileReader(minutesFile)

page = pdf1Reader.getPage(0)
page.rotateClockwise(90) # Aceita os valores 90, 180 e 270

pdfWriter = PyPDF2.PdfFileWriter()
pdfWriter.addPage(page)

resultPdfFile = open('data/rotatedPage.pdf', 'wb')
pdfWriter.write(resultPdfFile)

resultPdfFile.close()
minutesFile.close()

#### 4.3. Sobrepondo páginas

* Em Python, é fácil adicionar marcas-d'água em diversos arquivos e somente nas páginas especificadas pelo seu programa.


* Utilizaremos o método `mergePage()`.

In [13]:
minutesFile = open('data/meetingminutes.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(minutesFile)

minutesFirstPage = pdfReader.getPage(0)

pdfWatermarkReader = PyPDF2.PdfFileReader(open('data/watermark.pdf', 'rb'))

minutesFirstPage.mergePage(pdfWatermarkReader.getPage(0))

pdfWriter = PyPDF2.PdfFileWriter()
pdfWriter.addPage(minutesFirstPage)

for pageNum in range(1, pdfReader.numPages):
    pageObj = pdfReader.getPage(pageNum)
    pdfWriter.addPage(pageObj)
    
resultPdfFile = open('data/watermarkedCover.pdf', 'wb')
pdfWriter.write(resultPdfFile)

minutesFile.close()
resultPdfFile.close()

#### 4.4. Criptografando PDFs

* Antes de chamar o método `write()` para salvar os dados em um arquivo, chame o método `encrypt()` e passe uma string de senha a ele.

In [14]:
pdfFile = open('data/meetingminutes.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFile)

pdfWriter = PyPDF2.PdfFileWriter()

for pageNum in range(pdfReader.numPages):
    pdfWriter.addPage(pdfReader.getPage(pageNum))
    
pdfWriter.encrypt('swordfish')

resultPdf = open('data/encryptedminutes.pdf', 'wb')
pdfWriter.write(resultPdf)

resultPdf.close()